In [ ]:
import os
import numpy as np
from operator import add
import pandas as pd
import cv2
from google.colab import drive
import tensorflow as tf
from tensorflow import keras

In [ ]:
from functools import reduce

__all__ = ['Sgt', 'Episgt', 'Epiotrt']

ntmap = {'A': (1, 0, 0, 0),
         'C': (0, 1, 0, 0),
         'G': (0, 0, 1, 0),
         'T': (0, 0, 0, 1)
         }
epimap = {'A': 1, 'N': 0}


def get_seqcode(seq):
    return np.array(reduce(add, map(lambda c: ntmap[c], seq.upper()))).reshape(
        (1, len(seq), -1))


def get_epicode(eseq):
    return np.array(list(map(lambda c: epimap[c], eseq))).reshape(1, len(eseq), -1)


class Episgt:
    def __init__(self, fpath, num_epi_features, with_y=True):
        self._fpath = fpath
        self._ori_df = pd.read_csv(fpath, sep='\t', index_col=None, header=None)
        self._num_epi_features = num_epi_features
        self._with_y = with_y
        self._num_cols = num_epi_features + 2 if with_y else num_epi_features + 1
        self._cols = list(self._ori_df.columns)[-self._num_cols:]
        self._df = self._ori_df[self._cols]

    @property
    def length(self):
        return len(self._df)

    def get_dataset(self, x_dtype=np.float32, y_dtype=np.float32):
        x_seq = np.concatenate(list(map(get_seqcode, self._df[self._cols[0]])))
        x_epis = np.concatenate([np.concatenate(list(map(get_epicode, self._df[col]))) for col in
                                 self._cols[1: 1 + self._num_epi_features]], axis=-1)
        x = np.concatenate([x_seq, x_epis], axis=-1).astype(x_dtype)
        x = x.transpose(0, 2, 1)
        if self._with_y:
            y = np.array(self._df[self._cols[-1]]).astype(y_dtype)
            return x, y
        else:
            return x

In [ ]:
file_path = 'eg_cls_on_target.episgt'
input_data = Episgt(file_path, num_epi_features=4, with_y=True)
x, y = input_data.get_dataset()
x = np.expand_dims(x, axis=2)  # shape(x) = [100, 8, 1, 23]
np.shape(x)
# on_target_model_dir = 'ontar_ptaug_cnn'
# sess = tf.compat.v1.InteractiveSession()
# dcmodel = DCModelOntar(sess, on_target_model_dir, is_reg=False, seq_feature_only=False)
# predicted_on_target = dcmodel.ontar_predict(x)


(100, 8, 1, 23)

#### RUN UNDER run_example.py

In [ ]:
import numpy as np
import tensorflow as tf
import deepcrispr as dc
from tensorflow.contrib import slim
import sonnet as snt

## On-target Classification Task
file_path = 'examples/eg_cls_on_target.episgt'
input_data = dc.Episgt(file_path, num_epi_features=4, with_y=True)
x, y = input_data.get_dataset()
x = np.expand_dims(x, axis=2)  # shape(x) = [100, 8, 1, 23]

# Define the Saliency Map Calculation
def compute_saliency_map(model_output, input_tensor):
    saliency_map = tf.gradients(model_output, input_tensor)[0]
    saliency_map /= (tf.reduce_max(tf.abs(saliency_map)) + 1e-8)
    return saliency_map

# Session and Saver
sess = tf.InteractiveSession()
on_target_model_dir = 'trained_models/ontar_ptaug_cnn'
dcmodel = dc.DCModelOntar(sess, on_target_model_dir, is_reg=False, seq_feature_only=False)

# Assume 'dcmodel.pred_ontar' is the final output tensor of your model
model_output = dcmodel.pred_ontar

# Assume 'dcmodel.inputs_sg' is the input placeholder of your model
input_tensor = dcmodel.inputs_sg

# # Create a random input sample for testing
# input_sample = np.random.randn(1, 1, 23, 8)

# Compute the Saliency Map
saliency_map_tensor = compute_saliency_map(model_output, input_tensor)
x_transposed = np.transpose(x, (0, 2, 3, 1))

saliency_values = sess.run(saliency_map_tensor, feed_dict={input_tensor: x_transposed})
print(saliency_values)
print(np.shape(saliency_values))

# Visualize the Saliency Map
# plt.imshow(np.squeeze(saliency_values), cmap='hot')
# plt.show()

# # Close the session
# sess.close()


AttributeError: ignored

In [ ]:
def load_ckpt(sess, model_dir, variables_to_restore=None):
    ckpt = tf.train.get_checkpoint_state(model_dir)
    model_path = ckpt.model_checkpoint_path